In [4]:
import pickle
import os


MOVIES_DATA_FILE = 'movies04-01-2022-22-16-49-7497.pkl'

movies = pickle.load(open(os.path.join('data', MOVIES_DATA_FILE), 'rb'))
print('Number of movies', len(movies))

Number of movies 7497


In [14]:
from datetime import datetime

parsed_movies_scores = []
parsed_movies_release_dates = []
parsed_movies_titles = []
skipped_count = 0


for movie in movies:
    try:
        name = movie['page_title'].replace(' - Rotten Tomatoes', '')
        audience_score = int(movie['audience_score'])
        release_date = datetime.strptime(movie['release_date'], '%b %d, %Y')
        parsed_movies_release_dates.append(release_date)
        parsed_movies_scores.append(audience_score)
        parsed_movies_titles.append(name)
    except:
        skipped_count += 1

print('skipped', skipped_count)





skipped 66


In [15]:
from bokeh.plotting import figure, show, ColumnDataSource, output_file

source = ColumnDataSource(data=dict(
    x=parsed_movies_release_dates,
    y=parsed_movies_scores,
    parsed_movies_titles=parsed_movies_titles,
    parsed_movies_release_dates= [date.strftime("%m/%d/%Y") for date in parsed_movies_release_dates]
))

TOOLTIPS = [
    ("Title", "@parsed_movies_titles"),
    ("Audience Score", "$y"),
    ("Release Date", "@parsed_movies_release_dates"),
]

output_file(os.path.join('graphs', "movies.html"))
plot = figure(title="Movies by rotten tomatoes audience score", x_axis_label='Release Date', y_axis_label='Audience Score', x_axis_type='datetime',  tooltips=TOOLTIPS)

plot.circle('x', 'y', source=source)
show(plot)


